In [1]:
import pandas as pd
import numpy as np
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO

f = "axis02.pdf"

def came(f):
    try:
        tables=camelot.read_pdf(f,flavor='stream' ,pages = "all")
                 
        if len(tables) !=0 :
            df=pd.DataFrame();    tmp=pd.DataFrame(); 
            for i in range(len(tables)) :
                tmp=tables[i].df  
                if (tmp.shape[1] == 6 ):
                    df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                elif (tmp.shape[1] == 7):
                    tmp.drop(tmp.columns[[1]], axis=1, inplace=True)
                    tmp.columns = [0,1,2,3,4,5]
                    df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                else:
                    #print(tmp.shape[1])
                    print("\n different structure on page:",i); #log.write("\n different structure")        
    except:
        pass

    try:
        df.replace(to_replace = '', value =np.nan,inplace=True)
    except:pass

    return df

def tabu(f):
    tables=tabula.read_pdf(f,
                      pages='all',
                      silent=True,
                      multiple_tables=True,
                      pandas_options={'header':None})
    df = pd.DataFrame()
    df = pd.concat([c for c in tables]).drop_duplicates()
    return df

def proceed(df):
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            print("\nAxis-Column headers missing"); pass
        
    try:
        df = df.drop(["Init."], axis=1)
    except:
        try:
            df = df.drop(["Branch Name"], axis=1)
        except:pass
        
    df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(), axis=1) ==True].index)]
    df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)

    try:
        narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
    except:
        try:
            narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
        except:
            try:
                narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
            except:
                pass
            
    df = df[pd.notnull(df[narr])]

    df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

    def abc(a):
        if type(a) ==str:
            if len(a.split(' '))==2:
                z=a.split(' ')[1]
            else:
                z=a.split(' ')[0]
        else :
            z=a
        return z

    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
        df[bal]=df[bal].apply(lambda x: abc(x))
    except:
        print("\nBalance Column Missing")
        
    df["flag"]=df.iloc[:,-1].shift(1)
    df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

    df=df.T.drop_duplicates().T
    df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
    df["flag"]=df.iloc[:,0].astype(str)+df[bal].astype(str)
    df["flag2"]=np.arange(len(df))

    df.loc[df[["flag"]].duplicated(keep=False), 'flag'] = df["flag"]+df["flag2"].astype(str)
    df['flag']=df['flag'].apply(lambda row : np.nan if 'nannan' in row else row).fillna(method='bfill')

    df[narr]=df.groupby('flag')[narr].transform(lambda x:''.join(x))
    df=df.drop_duplicates(['flag'],keep='last').iloc[0:,:-2].reset_index(drop=True)
    df.to_csv("parsed.csv",index=0)


def fpage(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 1
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,caching=caching, check_extractable=False):
        interpreter.process_page(page)

    text = retstr.getvalue()
    #return text.lstrip().split("\n")

    ks=[c.upper() for c in text.lstrip().split("\n") [:50] if "" !=c ]
    
    lst=[]
    try:
        for j in ks:
            if "STATEMENT OF ACCOUNT NO" in j:
                index_no = ks.index(j)
                for k in ks[0:index_no+1]:
                    if k not in lst:
                        lst.append(k)
                return lst
            elif "STATEMENT OF AXIS ACCOUNT NO" in j:
                index_no = ks.index(j)
                for k in ks[0:index_no+1]:
                    if k not in lst:
                        lst.append(k)
                return lst
                
    except:
        pass                 

cst_info = fpage(f)

try:
    df = tabu(f); df = proceed(df)
    print("Completed...1")
except:
    df = came(f); df = proceed(df)
    print("Completed...2")

print(cst_info)

Completed...1
['TRAN DATE', 'VALUE DATE', 'TRANSACTION PARTICULARS', 'CHQ NO', 'AMOUNT(INR)', 'DR/CR', 'BALANCE(INR) BRANCH NAME', 'STATEMENT OF AXIS ACCOUNT NO :910020037211732 FOR THE PERIOD (FROM : 01-11-2017  TO : 31-10-2018)']
